In [1]:
import csv
import cv2
import numpy as np
import tensorflow as tf
import time
from math import atan2,cos,sin

class HandTracker():
    
    def __init__(self, palm_model, joint_model, anchors_path,
                box_enlarge=1.5, box_shift=0.2):
        self.box_shift = box_shift
        self.box_enlarge = box_enlarge

        self.interp_palm = tf.lite.Interpreter(palm_model)
        self.interp_palm.allocate_tensors()
        self.interp_joint = tf.lite.Interpreter(joint_model)
        self.interp_joint.allocate_tensors()

        # reading the SSD anchors
        with open(anchors_path, "r") as csv_f:
            self.anchors = np.r_[
                [x for x in csv.reader(csv_f, quoting=csv.QUOTE_NONNUMERIC)]
            ]
        # reading tflite model paramteres
        output_details = self.interp_palm.get_output_details()
        input_details = self.interp_palm.get_input_details()

        self.in_idx = input_details[0]['index']
        self.out_reg_idx = output_details[0]['index']
        self.out_clf_idx = output_details[1]['index']

        self.in_idx_joint = self.interp_joint.get_input_details()[0]['index']
        self.out_idx_joint = self.interp_joint.get_output_details()[0]['index']

        # 90° rotation matrix used to create the alignment triangle
        self.R90 = np.r_[[[0,1],[-1,0]]]

        # trianlge target coordinates used to move the detected hand
        # into the right position
        self._target_triangle = np.float32([
                        [128, 128],
                        [128,   0],
                        [  0, 128]
                    ])
        self._target_box = np.float32([
                        [  0,   0, 1],
                        [256,   0, 1],
                        [256, 256, 1],
                        [  0, 256, 1],
                    ])

    def _get_triangle(self, kp0, kp2, dist=1):
        """get a triangle used to calculate Affine transformation matrix"""

        dir_v = kp2 - kp0
        dir_v /= np.linalg.norm(dir_v)

        dir_v_r = dir_v @ self.R90.T
        return np.float32([kp2, kp2+dir_v*dist, kp2 + dir_v_r*dist])

    @staticmethod
    def _triangle_to_bbox(source):
        # plain old vector arithmetics
        bbox = np.c_[
            [source[2] - source[0] + source[1]],
            [source[1] + source[0] - source[2]],
            [3 * source[0] - source[1] - source[2]],
            [source[2] - source[1] + source[0]],
        ].reshape(-1,2)
        return bbox

    @staticmethod
    def _im_normalize(img):
         return np.ascontiguousarray(2 * ((img / 255) - 0.5).astype('float32'))

    @staticmethod
    def _sigm(x):
        return 1 / (1 + np.exp(-x) )

    @staticmethod
    def _pad1(x):
        return np.pad(x, ((0,0),(0,1)), constant_values=1, mode='constant')


    def predict_joints(self, img_norm):
        self.interp_joint.set_tensor(
            self.in_idx_joint, img_norm.reshape(1,256,256,3))
        self.interp_joint.invoke()

        joints = self.interp_joint.get_tensor(self.out_idx_joint)
        return joints.reshape(-1,2)

    def detect_hand(self, img_norm):
        assert -1 <= img_norm.min() and img_norm.max() <= 1,\
        "img_norm should be in range [-1, 1]"
        assert img_norm.shape == (256, 256, 3),\
        "img_norm shape must be (256, 256, 3)"

        # predict hand location and 7 initial landmarks
        self.interp_palm.set_tensor(self.in_idx, img_norm[None])
        self.interp_palm.invoke()

        out_reg = self.interp_palm.get_tensor(self.out_reg_idx)[0]
        out_clf = self.interp_palm.get_tensor(self.out_clf_idx)[0,:,0]
        
        detecion_mask = self._sigm(out_clf) > 0.7
        candidate_detect = out_reg[detecion_mask]
        candidate_anchors = self.anchors[detecion_mask]
        ar=[]
        for i in range(len(detecion_mask)):
            if(detecion_mask[i]):
                ar.append(i)
        
        if candidate_detect.shape[0] == 0:
#             print("No hands found")
            return None, None
        max_idx = np.argmax(candidate_detect[:, 3])

        dx,dy,w,h = candidate_detect[max_idx, :4]
        center_wo_offst = candidate_anchors[max_idx,:2] * 256
        keypoints = center_wo_offst + candidate_detect[max_idx,4:].reshape(-1,2)
        side = max(w,h) * self.box_enlarge
        source = self._get_triangle(keypoints[0], keypoints[2], side)
        source -= (keypoints[0] - keypoints[2]) * self.box_shift
        return source, keypoints

    def preprocess_img(self, img):
        # fit the image into a 256x256 square
        shape = np.r_[img.shape]
        pad = (shape.max() - shape[:2]).astype('uint32') // 2
        img_pad = np.pad(
            img,
            ((pad[0],pad[0]), (pad[1],pad[1]), (0,0)),
            mode='constant')
        img_small = cv2.resize(img_pad, (256, 256))
        img_small = np.ascontiguousarray(img_small)

        img_norm = self._im_normalize(img_small)
        return img_pad, img_norm, pad


    def __call__(self, img):
        img_pad, img_norm, pad = self.preprocess_img(img)

        st=time.time()
        source, keypoints = self.detect_hand(img_norm)
        end = time.time()
        if source is None:
            return None, None, None

        scale = max(img.shape) / 256
        Mtr = cv2.getAffineTransform(
            source * scale,
            self._target_triangle
        )
        padd=self._im_normalize(img_pad)
        img_landmark = cv2.warpAffine(
            padd, Mtr, (256,256)
        )
        joints = self.predict_joints(img_landmark)

        Mtr = self._pad1(Mtr.T).T
        Mtr[2,:2] = 0

        Minv = np.linalg.inv(Mtr)

        kp_orig = (self._pad1(joints) @ Minv.T)[:,:2]
        box_orig = (self._target_box @ Minv.T)[:,:2]
        kp_orig -= pad[::-1]
        box_orig -= pad[::-1]
        
        return kp_orig, box_orig, img_landmark

In [ ]:
import tensorflow as tf

pd = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(pd[0],True)

%run "Gesture Prediction with Saimese.ipynb"

WINDOW = "Hand Tracking"
PALM_MODEL_PATH = "garbage/models/palm_detector.tflite"
LANDMARK_MODEL_PATH = "garbage/models/hand_landmark.tflite"
ANCHORS_PATH = "garbage/anchors.csv"

POINT_COLOR = (0, 255, 0)
CONNECTION_COLOR = (255, 0, 0)
THICKNESS = 2

cv2.namedWindow(WINDOW)
capture = cv2.VideoCapture(0)

if capture.isOpened():
    hasFrame, frame = capture.read()
    frame = cv2.flip(frame,1)
    forcomp=frame
else:
    hasFrame = False

connections = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

detector = HandTracker(
    PALM_MODEL_PATH,
    LANDMARK_MODEL_PATH,
    ANCHORS_PATH,
    box_shift=0.2,
    box_enlarge=1.2
)

with open('garbage/GestureProject/GestureLabels.txt',"r") as labelFile:
    import json
    labels = json.load(labelFile)
print(labels)

rotateit = lambda point: (int(point[0]*cos(ar)-point[1]*sin(ar)),int(point[0]*sin(ar)+point[1]*cos(ar)))
handlandmarks = []
counter = 0
reqlen = 0.2
trainData = True
p = PredictLandmarks(ModelPath="garbage/GestureProject/models/GesSaimeseModel/iteration2",CompareGesPath="garbage/GestureProject/RawGestures/RelocatedRawGes10R.csv",trainData=trainData)

while hasFrame:
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    points, box, source = detector(image)
    
    if points is not None:
        
        for point in points:
            x, y = point
            cv2.circle(frame, (int(x), int(y)), THICKNESS * 2, POINT_COLOR, THICKNESS)
            
        
        for connection in connections:
            x0, y0 = points[connection[0]]
            x1, y1 = points[connection[1]]
            cv2.line(frame, (int(x0), int(y0)), (int(x1), int(y1)), CONNECTION_COLOR, THICKNESS)
        
        
#         if p is defined the only predict and show result as well as landmark
        if 'p' in vars():
            prep = preprocessLm(points, normalize=True, trainData=trainData)
            label = labels[str(p.predict(prep))]
            blackBackground = np.zeros((480,640,3))
            xPoint = prep.reshape((21,2))*(480,640)
            cv2.putText(frame, label, (30,30), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0,0,255),1)

            for point in xPoint:
                x, y = point
                cv2.circle(blackBackground, (int(x), int(y)), THICKNESS * 2, POINT_COLOR, THICKNESS)

            for connection in connections:
                x0, y0 = xPoint[connection[0]]
                x1, y1 = xPoint[connection[1]]
                cv2.line(blackBackground, (int(x0), int(y0)), (int(x1), int(y1)), CONNECTION_COLOR, THICKNESS)

            cv2.imshow("Feeded data", blackBackground)
        
        
    cv2.imshow(WINDOW, frame)
    hasFrame, frame = capture.read()
    frame = cv2.flip(frame,1)
    key = cv2.waitKey(1)
    if key == 121:
        handlandmarks.append(points)
        print('coords added')
    if key == 27:
        break
capture.release()
cv2.destroyAllWindows()

if len(handlandmarks) is not 0:
    key = input("press y to save Coordinates: ")
    if key == 'y' or key == 'Y':
        basePath = "garbage/GestureProject/TestingCoordinates/"
        fileName = input('Enter Input file name')
        filePath = basePath + fileName + '.csv'
        print("Landmaks saved at [" + filePath + "]")
        import pandas as pd
        lmarray = np.asarray(handlandmarks)
        frame = pd.DataFrame(lmarray.reshape((-1,42)))
        frame.to_csv(filePath, index=False)

{'0': 'One', '1': 'Two', '2': 'Three', '3': 'Four', '4': 'Five', '5': 'Tasty', '6': 'SpiderMan', '7': 'SuperMan', '8': 'ThumbsUp', '9': 'ThumbsDown'}
[normalizeLm]: Got landmarks of shape 42. Reshaping to (21,2)
[normalizeLm]: Got landmarks of shape 42. Reshaping to (21,2)


C:\Users\Assessioner\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: overflow encountered in exp


In [14]:
# labels = {
#         0:"One",
#         1:"Two",
#         2:"Three",
#         3:"Four",
#         4:"Five",
#         5:"Tasty",
#         6:"SpiderMan",
#         7:"SuperMan",
#         8:"ThumbsUp"
# }